# SETUP

In [1]:
!git clone https://github.com/Yichuan0712/rla.git

Cloning into 'rla'...
remote: Enumerating objects: 320, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 320 (delta 53), reused 72 (delta 25), pack-reused 209
Receiving objects: 100% (320/320), 39.92 MiB | 33.18 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [2]:
import os
os.chdir('rla')
os.listdir(os.getcwd())

['example_data',
 'clip_main.py',
 'contact_head',
 '__pycache__',
 'example_notebooks',
 'README.md',
 '.git',
 'images',
 'terminator_utils',
 'terminator_configs',
 'dataset_configs',
 '.idea',
 'pdb_blacklist.txt',
 'src',
 'terminator',
 'rla_env.yml']

In [3]:
!pip install open_clip_torch

import torch
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

!pip install fastargs
!pip install terminaltables
!pip install biopython
!pip install webdataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x8

In [4]:
import os
import src.models_and_optimizers as model_utils
import yaml
from types import SimpleNamespace
from clip_main import get_wds_loaders
from transformers import EsmTokenizer
import src.data_utils as data_utils
import torch
import sys
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import json
from torch.cuda.amp import autocast
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import webdataset as wds
import copy

<ipython-input-4-264f5a970416>:16: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [5]:
# ## Load util functions
# # os.chdir('example_notebooks')
# !pip install tmtools
# !pip install tmscoring
# !pip install iminuit
# # %run example_notebooks/rla_utils.ipynb
# # os.chdir('..')

In [6]:

# import os
# import src.models_and_optimizers as model_utils
# import yaml
# from types import SimpleNamespace
# from clip_main import get_wds_loaders
# from transformers import EsmTokenizer
# import src.data_utils as data_utils
# import torch
# import sys
# import pickle
# from tqdm import tqdm
# import numpy as np
# from tmtools.io import get_structure, get_residue_data
# from tmtools import tm_align
# import matplotlib.pyplot as plt
# import json
# from torch.cuda.amp import autocast
# import tmscoring
# import json
# import copy
# from scipy.stats.stats import pearsonr
# import matplotlib.pyplot as plt
# import yaml
# import pandas as pd
# import glob
# import webdataset as wds

In [7]:
!git pull origin main

From https://github.com/Yichuan0712/rla
 * branch            main       -> FETCH_HEAD
Already up to date.


In [8]:
!cp /content/drive/MyDrive/RLA -r /content/rla

# CONVERT

In [47]:
import numpy as np
import gzip
import warnings
from pathlib import Path
from Bio.PDB.PDBExceptions import PDBConstructionWarning
from Bio.PDB import PDBParser
from Bio import SeqIO
from Bio.SeqUtils import seq1
import webdataset as wds
import sys
import torch
import numpy as np
import os
import tqdm
import pandas as pd
from datetime import datetime
import argparse
import tarfile


def write_dataset(dataset, tar_name, use_shards=False, max_shard_count=10000):
    if use_shards:
        os.makedirs(tar_name, exist_ok=True)
        sink = wds.ShardWriter(f'{tar_name}/shard-%06d.tar',maxcount=max_shard_count)
    else:
        sink = wds.TarWriter(tar_name)
    for index, (batch, pdb_id) in enumerate(dataset):
        if index%1000==0:
            print(f"{index:6d}", end="\r", flush=True, file=sys.stderr)
        if len(batch[0]) == 0:
            continue
        sink.write({
            "__key__": "sample%06d" % index,
            "inp.pyd": dict(coords=batch[0], seqs=batch[1], chain_ids=batch[2], pdb_id=pdb_id),
        })
    sink.close()


def process_residue(residue):
    atoms = ['N', 'CA', 'C', 'O']
    coordinates = []
    for r in atoms:
        coord = residue.child_dict.get(r, None)
        if coord is None:
            if r == 'O':
                coord = residue.child_dict.get('OXT', None)
            if coord is None:
                return None, None
        coordinates.append(np.array(coord.get_coord()))
    return np.stack(coordinates), seq1(residue.resname)


def process_chain(chain):
    coordinates = []
    seq = []
    for r in chain:
        output, residue_name = process_residue(r)
        if output is not None:
            coordinates.append(output)
            seq.append(residue_name)
    if len(coordinates) == 0:
        return None
    coordinates = np.stack(coordinates)
    seq = ''.join(seq)
    return coordinates, seq


def process_chains(chains):
    output = []
    chain_ids = []
    for chain in chains:
        out = process_chain(chain)
        if out is not None:
            output.append(out)
            chain_ids.append(chain.id)
    coords = [u[0] for u in output]
    seqs = [u[1] for u in output]
    return coords, seqs, chain_ids


def process_structure(structure):
    for s in structure: # only one structure
        return process_chains(s)
    return None


def process_pdb(parser, pdb_filename):
    # print(pdb_filename)
    with open(pdb_filename, "r") as file_handle:
        structure = parser.get_structure("?", file_handle)
        date = structure.header['deposition_date']
        return process_structure(structure), date


def write_pdb_datasets():
    parser = PDBParser(QUIET=True)
    root_pdb = "/content/rla/RLA/AlphaFold_model_PDBs"
    pdb_dir = [os.path.join(root_pdb, u) for u in os.listdir(root_pdb)]

    output = []
    for pdb_filename in pdb_dir:
        # print(pdb_filename)
        pdb_id = os.path.basename(pdb_filename).split('.')[0]  # .split('pdb')[1]
        # print(pdb_filename)
        out, date = process_pdb(parser, pdb_filename)
        # print(date)
        if out is None:
            continue
        # print(out)
        output.append((out, pdb_id))

    destination_path='/content/rla/RLA/example_data/dataset.tar'
    write_dataset(output, destination_path)


write_pdb_datasets()

In [48]:
!cp /content/rla/RLA/example_data/dataset.tar /content/drive/MyDrive/RLA/example_data